In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Пробуем ускорить классификатор расы для лиц из работы 5.

In [44]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

import os
import tensorflow as tf
print(tf.__version__)

import sys
import keras
import keras.backend as K
import json
np.random.seed(42)
import pickle

2.3.0


In [45]:
! wget https://img.joinfo.ua/i/2019/01/5c4ea940d2b08.jpg -O brad_pitt.jpg

--2020-12-03 00:22:48--  https://img.joinfo.ua/i/2019/01/5c4ea940d2b08.jpg
Resolving img.joinfo.ua (img.joinfo.ua)... 172.67.74.175, 104.26.7.169, 104.26.6.169, ...
Connecting to img.joinfo.ua (img.joinfo.ua)|172.67.74.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43154 (42K) [image/jpeg]
Saving to: ‘brad_pitt.jpg’

brad_pitt.jpg       100%[===================>]  42.14K  --.-KB/s    in 0.01s   

2020-12-03 00:22:50 (3.18 MB/s) - ‘brad_pitt.jpg’ saved [43154/43154]



In [46]:
!pip install keras_applications

     |████████████████████████████████| 51kB 2.6MB/s 


In [48]:
! pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-22we9a3r
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-22we9a3r
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=cf2bc1d655f6561c7065855f42762088536b52bdf854a5298d525c7b98575b29
  Stored in directory: /tmp/pip-ephem-wheel-cache-64hlv9kd/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [64]:
from keras_vggface.vggface import VGGFace
from tensorflow.keras.preprocessing import image
from keras_vggface import utils

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224)) # модель работает с картинками размера 224 на 224
    x = np.expand_dims(image.img_to_array(img), axis=0)
    return utils.preprocess_input(x, version=2)

preprocess_image("./brad_pitt.jpg")

array([[[[ -65.4953  ,  -86.8827  , -117.0912  ],
         [ -65.4953  ,  -86.8827  , -117.0912  ],
         [ -67.4953  ,  -88.8827  , -119.0912  ],
         ...,
         [ -71.4953  ,  -88.8827  , -117.0912  ],
         [ -71.4953  ,  -88.8827  , -117.0912  ],
         [ -73.4953  ,  -90.8827  , -119.0912  ]],

        [[ -64.4953  ,  -85.8827  , -116.0912  ],
         [ -64.4953  ,  -85.8827  , -116.0912  ],
         [ -66.4953  ,  -87.8827  , -118.0912  ],
         ...,
         [ -71.4953  ,  -88.8827  , -117.0912  ],
         [ -71.4953  ,  -88.8827  , -117.0912  ],
         [ -71.4953  ,  -88.8827  , -117.0912  ]],

        [[ -61.4953  ,  -82.8827  , -113.0912  ],
         [ -61.4953  ,  -82.8827  , -113.0912  ],
         [ -63.4953  ,  -84.8827  , -115.0912  ],
         ...,
         [ -71.4953  ,  -88.8827  , -117.0912  ],
         [ -71.4953  ,  -88.8827  , -117.0912  ],
         [ -72.4953  ,  -89.8827  , -118.0912  ]],

        ...,

        [[ 139.5047  ,  123.1173  ,   

In [51]:
converter = trt.TrtGraphConverterV2(input_saved_model_dir='./drive/My Drive/models/model_gender/saved_model', )
converter.convert()
converter.save('./drive/My Drive/models/model_gender/saved_model_trt')

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Assets written to: ./drive/My Drive/models/model_gender/saved_model_trt/assets


In [52]:
model1 = tf.saved_model.load("./drive/My Drive/models/model_gender/saved_model")
model2 = tf.saved_model.load("./drive/My Drive/models/model_gender/saved_model_trt")

In [55]:
test_image = preprocess_image("./brad_pitt.jpg")

In [60]:
%%timeit -n 20 -r 20

preds = model1(test_image)

20 loops, best of 20: 161 ms per loop


In [66]:
empty = np.zeros((1, 224, 224, 3), dtype="float32")
empty

array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)

In [61]:
%%timeit -n 20 -r 20

preds = model2(test_image)

20 loops, best of 20: 165 ms per loop


In [67]:
%%timeit -n 10 -r 10

preds = model1(empty)

10 loops, best of 10: 165 ms per loop


In [69]:
%%timeit -n 10 -r 10

preds = model2(empty)

10 loops, best of 10: 168 ms per loop


In [ ]:
#Вывод - TensorRT работает даже в сторону ухудшения на больших картиночных моделях.........